# Importing the libraries

# New Section

In [ ]:
!git clone https://github.com/mandar196/Hate_Speech_Detection-NLP.git

Cloning into 'Hate_Speech_Detection-NLP'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 19 (delta 4), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (19/19), done.


In [ ]:
!gdown --id 1ti6QbZh_j9rh9fhKGZIoThM7FN9tF3Un #training data

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1ti6QbZh_j9rh9fhKGZIoThM7FN9tF3Un
To: /content/labeled_data.csv
100% 2.57M/2.57M [00:00<00:00, 113MB/s]


In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from wordcloud import WordCloud

#to data preprocessing
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

#NLP tools
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer

#train split and fit models
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB

#model selection
from sklearn.metrics import confusion_matrix, accuracy_score


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# Importing the dataset

In [2]:
dataset = pd.read_csv('/content/stormfront1.csv')
dataset.head()

,Unnamed: 0,body,gold_label,User Id
0,0,Fuck that I dont wanna watch tranny porn.,1.0,0
1,1,Opie just wanted to have a good time. The same...,1.0,1
2,2,Fuck that faggot Fallon. Fucking sissy boy act...,1.0,2
3,3,BbBB...b.b..b.bb but OP's a faggot,1.0,3
4,4,Who even uses the word tranny except for trans...,0.0,4


In [3]:
dataset.fillna('None')
dataset["body"].fillna("No Tweet", inplace = True)
dataset["gold_label"].fillna(0, inplace = True)

In [4]:
print(dataset['gold_label'].unique())

[1. 0.]


In [5]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40003 entries, 0 to 40002
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  40003 non-null  int64  
 1   body        40003 non-null  object 
 2   gold_label  40003 non-null  float64
 3   User Id     40003 non-null  int64  
dtypes: float64(1), int64(2), object(1)
memory usage: 1.2+ MB


In [6]:
dataset.describe().T

,count,mean,std,min,25%,50%,75%,max
Unnamed: 0,40003.0,20001.000000,11548.015746,0.0,10000.5,20001.0,30001.5,40002.0
gold_label,40003.0,0.513212,0.499832,0.0,0.0,1.0,1.0,1.0
User Id,40003.0,20001.000000,11548.015746,0.0,10000.5,20001.0,30001.5,40002.0


In [7]:
# dataset['Class'] = dataset['Class'].replace({'sexism': 0})
# dataset['Class'] = dataset['Class'].replace({'none': 1})
# dataset['Class'] = dataset['Class'].replace({'racism': 0})

In [8]:
dicti = { 'gold_label': 'class','body': 'tweet'}
dataset.rename(columns=dicti,
          inplace=True)

In [9]:
dt_trasformed = dataset[['class', 'tweet']]
y = dt_trasformed.iloc[:, :-1].values
print(y)

[[1.]
 [1.]
 [1.]
 ...
 [0.]
 [0.]
 [0.]]


### Encoding the Dependent Variable

In [10]:
# ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [0])], remainder='passthrough')
# y = np.array(ct.fit_transform(y))

In [11]:
print(y)

[[1.]
 [1.]
 [1.]
 ...
 [0.]
 [0.]
 [0.]]


I separated this y in two variables that we will use to fit hate speech models and offensive speech models

In [12]:
# y_df = pd.DataFrame(y)
# y_hate = np.array(y_df[0])
# y_offensive = np.array(y_df[1])
# y_neither = np.array(y_df[2])
# print(sum(y_neither==2))

In [13]:
#extra
xy = np.ravel(y,order='C')

In [14]:
# print(y_hate)
# print(y_offensive)

## Cleaning the texts

In [15]:
corpus = []
for i in range(0, 40003):
  review = re.sub('[^a-zA-Z]', ' ', dt_trasformed['tweet'][i])
  review = review.lower()
  review = review.split()
  ps = PorterStemmer()
  all_stopwords = stopwords.words('english')
  all_stopwords.remove('not')
  review = [ps.stem(word) for word in review if not word in set(all_stopwords)]
  review = ' '.join(review)
  corpus.append(review)

In [16]:
cv = CountVectorizer(max_features = 2000)
X = cv.fit_transform(corpus).toarray()

## Splitting the dataset into the Training set and Test set

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, xy, test_size = 0.30, random_state = 0,shuffle=True)

In [18]:
# dfd = pd.DataFrame(y_test)
# dfd.to_csv('ML_RE_ytest.csv')
# # print(type(data))
# dfdd = pd.DataFrame(X_test)
# dfdd.to_csv('ML_RE_Xtest.csv')

## Finding the best models to predict hate speech

**Naive Bayes**

In [19]:
classifier_np = GaussianNB()
classifier_np.fit(X_train, y_train)

GaussianNB()

**Decision Tree**

In [20]:
classifier_dt = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
classifier_dt.fit(X_train, y_train)

DecisionTreeClassifier(criterion='entropy', random_state=0)

**KNN**

In [21]:
classifier_knn = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
classifier_knn.fit(X_train, y_train)

KNeighborsClassifier()

**Logistic Regression**

In [22]:
classifier_lr = LogisticRegression(random_state = 0)
classifier_lr.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression(random_state=0)

**Random Forest**

In [23]:
classifier_rf = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
classifier_rf.fit(X_train, y_train)

RandomForestClassifier(criterion='entropy', n_estimators=10, random_state=0)

SVM Classifier

In [24]:
classifier_svm = svm.SVC()
classifier_svm.fit(X_train, y_train)

SVC()

XGBoost Classifier

In [25]:
classifier_xgb= XGBClassifier()
classifier_xgb.fit(X_train, y_train)

XGBClassifier()

# Making the Confusion Matrix for each model

In [26]:
from sklearn.metrics import classification_report

In [27]:
#Naive Bayes
y_pred_np = classifier_np.predict(X_test)
cm = confusion_matrix(y_test, y_pred_np)
print(classification_report(y_test,y_pred_np))
print(y_pred_np)
print(cm)


              precision    recall  f1-score   support

         0.0       0.72      0.53      0.61      5813
         1.0       0.65      0.81      0.72      6188

    accuracy                           0.68     12001
   macro avg       0.69      0.67      0.67     12001
weighted avg       0.68      0.68      0.67     12001

[1. 1. 0. ... 1. 0. 1.]
[[3093 2720]
 [1178 5010]]


In [28]:
#KNN
y_pred_knn = classifier_knn.predict(X_test)
cm = confusion_matrix(y_test, y_pred_knn)
print(classification_report(y_test,y_pred_knn))
print(cm)

              precision    recall  f1-score   support

         0.0       0.86      0.27      0.41      5813
         1.0       0.58      0.96      0.73      6188

    accuracy                           0.63     12001
   macro avg       0.72      0.62      0.57     12001
weighted avg       0.72      0.63      0.57     12001

[[1585 4228]
 [ 255 5933]]


In [29]:
#XGBoost Classifier
y_pred_xgb = classifier_xgb.predict(X_test)
cm = confusion_matrix(y_test, y_pred_xgb)
print(classification_report(y_test,y_pred_xgb))
print(cm)

              precision    recall  f1-score   support

         0.0       0.81      0.70      0.75      5813
         1.0       0.75      0.84      0.79      6188

    accuracy                           0.77     12001
   macro avg       0.78      0.77      0.77     12001
weighted avg       0.78      0.77      0.77     12001

[[4068 1745]
 [ 967 5221]]


In [30]:
#SVM
y_pred_svm = classifier_svm.predict(X_test)
cm = confusion_matrix(y_test, y_pred_svm)
print(classification_report(y_test,y_pred_svm))
print(cm)

              precision    recall  f1-score   support

         0.0       0.80      0.79      0.80      5813
         1.0       0.81      0.81      0.81      6188

    accuracy                           0.80     12001
   macro avg       0.80      0.80      0.80     12001
weighted avg       0.80      0.80      0.80     12001

[[4615 1198]
 [1176 5012]]


In [31]:
#Logistic Regression
y_pred_lr=classifier_lr.predict(X_test)
cm = confusion_matrix(y_test, y_pred_lr)
print(classification_report(y_test,y_pred_lr))
print(cm)

              precision    recall  f1-score   support

         0.0       0.80      0.75      0.78      5813
         1.0       0.78      0.82      0.80      6188

    accuracy                           0.79     12001
   macro avg       0.79      0.79      0.79     12001
weighted avg       0.79      0.79      0.79     12001

[[4382 1431]
 [1093 5095]]


In [32]:
#Decision Tree
y_pred_dt = classifier_dt.predict(X_test)
cm = confusion_matrix(y_test, y_pred_dt)
print(classification_report(y_test,y_pred_dt))
print(cm)


              precision    recall  f1-score   support

         0.0       0.73      0.71      0.72      5813
         1.0       0.73      0.75      0.74      6188

    accuracy                           0.73     12001
   macro avg       0.73      0.73      0.73     12001
weighted avg       0.73      0.73      0.73     12001

[[4133 1680]
 [1557 4631]]


In [33]:
#Random Florest
y_pred_rf = classifier_rf.predict(X_test)
cm = confusion_matrix(y_test, y_pred_rf)
print(classification_report(y_test,y_pred_rf))
print(cm)

              precision    recall  f1-score   support

         0.0       0.75      0.78      0.76      5813
         1.0       0.78      0.75      0.77      6188

    accuracy                           0.76     12001
   macro avg       0.76      0.77      0.76     12001
weighted avg       0.77      0.76      0.76     12001

[[4511 1302]
 [1522 4666]]


In [34]:
rf_score = accuracy_score(y_test, y_pred_rf)
knn_score = accuracy_score(y_test, y_pred_knn)
svm_score = accuracy_score(y_test, y_pred_svm)
xgb_score = accuracy_score(y_test, y_pred_xgb)
lr_score = accuracy_score(y_test, y_pred_lr)
dt_score = accuracy_score(y_test, y_pred_dt)
np_score = accuracy_score(y_test, y_pred_np)

print('Random Forest Accuracy: ', str(rf_score))
print('K Nearest Neighbours Accuracy: ', str(knn_score))
print('Support Vector Machine Accuracy: ', str(svm_score))
print('XGBoost Classifier Accuracy: ', str(xgb_score))
print('Logistic Regression Accuracy: ',str(lr_score))
print('Decision Tree Accuracy: ', str(dt_score))
print('Naive Bayes Accuracy: ', str(np_score))

Random Forest Accuracy:  0.7646862761436547
K Nearest Neighbours Accuracy:  0.6264477960169986
Support Vector Machine Accuracy:  0.8021831514040496
XGBoost Classifier Accuracy:  0.7740188317640196
Logistic Regression Accuracy:  0.789684192983918
Decision Tree Accuracy:  0.7302724772935588
Naive Bayes Accuracy:  0.6751937338555121


So XGBoost Classifier looks better to predict hate speech based on this dataset. It's important to emphasize Random Forest,KNN and Decision Tree had great results as well. This Dataset looks like a product of artificial intelligence to classify hate and offensive speech.

In [35]:
#Now finally saving the model with XGB Classifier.
import joblib
joblib.dump(classifier_xgb, 'hatespeech.pkl')

['hatespeech.pkl']

In [36]:
from sklearn.metrics import precision_recall_fscore_support

In [37]:
score_rf=precision_recall_fscore_support(y_test,y_pred_rf,average='weighted')
print(score_rf)

(0.7653116765573986, 0.7646862761436547, 0.7647419997500496, None)


In [38]:
score_knn=precision_recall_fscore_support(y_test,y_pred_knn,average='weighted')
print(score_knn)

(0.7183203408613638, 0.6264477960169986, 0.5748731214326025, None)


In [39]:
score_svm=precision_recall_fscore_support(y_test,y_pred_svm,average='weighted')
print(score_svm)

(0.8021645279735785, 0.8021831514040496, 0.8021711421200771, None)


In [40]:
score_xgb=precision_recall_fscore_support(y_test,y_pred_xgb,average='weighted')
print(score_xgb)

(0.7778078188073158, 0.7740188317640196, 0.7725982257801054, None)


In [41]:
score_lr=precision_recall_fscore_support(y_test,y_pred_lr,average='weighted')
print(score_lr)

(0.790237397564223, 0.789684192983918, 0.7893310269427718, None)


In [42]:
score_dt=precision_recall_fscore_support(y_test,y_pred_dt,average='weighted')
print(score_dt)

(0.7301962637059506, 0.7302724772935588, 0.730157563190923, None)


In [43]:
score_np=precision_recall_fscore_support(y_test,y_pred_np,average='weighted')
print(score_np)

(0.6849669097794283, 0.6751937338555121, 0.6683527082421057, None)


# Testing

In [44]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## Test on Large Facebook

In [45]:
d = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/ML_TEST/Data/ML_FB!_Xtest.csv')
# d.head(5)
d.columns.values[0] = 'id'
d.drop('id', inplace=True, axis=1)
X_t = d.to_numpy()
de = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/ML_TEST/Data/ML_FB!_ytest.csv')
# d.head(5)
de.columns.values[0] = 'id'
de.drop('id', inplace=True, axis=1)
y_t = de.to_numpy()
# X_t

In [46]:
#Naive Bayes by Davidson or itself
y_pred_np = classifier_np.predict(X_t)
cm = confusion_matrix(y_t, y_pred_np)
print(classification_report(y_t,y_pred_np))
print(cm)
score_np=precision_recall_fscore_support(y_t,y_pred_np,average='weighted')
print(score_np)
score_np=precision_recall_fscore_support(y_t,y_pred_np,average='micro')
print(score_np)
score_np=precision_recall_fscore_support(y_t,y_pred_np,average='macro')
print(score_np)

              precision    recall  f1-score   support

           0       0.40      0.43      0.41       929
           1       0.66      0.63      0.64      1621

    accuracy                           0.56      2550
   macro avg       0.53      0.53      0.53      2550
weighted avg       0.56      0.56      0.56      2550

[[ 398  531]
 [ 602 1019]]
(0.5629093232131562, 0.5556862745098039, 0.5588889372605046, None)
(0.5556862745098039, 0.5556862745098039, 0.5556862745098039, None)
(0.5277096774193548, 0.5285209796873516, 0.5276742524226895, None)


In [47]:
#KNN
y_pred_knn = classifier_knn.predict(X_t)
cm = confusion_matrix(y_t, y_pred_knn)
print(classification_report(y_t,y_pred_knn))
print(cm)
score_knn=precision_recall_fscore_support(y_t,y_pred_knn,average='weighted')
print(score_knn)
score_knn=precision_recall_fscore_support(y_t,y_pred_knn,average='micro')
print(score_knn)
score_knn=precision_recall_fscore_support(y_t,y_pred_knn,average='macro')
print(score_knn)

              precision    recall  f1-score   support

           0       0.39      0.03      0.06       929
           1       0.64      0.97      0.77      1621

    accuracy                           0.63      2550
   macro avg       0.51      0.50      0.41      2550
weighted avg       0.55      0.63      0.51      2550

[[  29  900]
 [  46 1575]]
(0.5453956030897207, 0.6290196078431373, 0.5099160717729572, None)
(0.6290196078431373, 0.6290196078431373, 0.6290196078431373, None)
(0.5115151515151515, 0.5014194084768734, 0.4134059465263944, None)


In [48]:
#XGBoost Classifier
y_pred_xgb = classifier_xgb.predict(X_t)
cm = confusion_matrix(y_t, y_pred_xgb)
print(classification_report(y_t,y_pred_xgb))
print(cm)
score_xgb=precision_recall_fscore_support(y_t,y_pred_xgb,average='weighted')
print(score_xgb)
score_xgb=precision_recall_fscore_support(y_t,y_pred_xgb,average='micro')
print(score_xgb)
score_xgb=precision_recall_fscore_support(y_t,y_pred_xgb,average='macro')
print(score_xgb)

              precision    recall  f1-score   support

           0       0.48      0.02      0.03       929
           1       0.64      0.99      0.78      1621

    accuracy                           0.64      2550
   macro avg       0.56      0.50      0.40      2550
weighted avg       0.58      0.64      0.50      2550

[[  14  915]
 [  15 1606]]
(0.5808387637154192, 0.6352941176470588, 0.5036041359444496, None)
(0.6352941176470588, 0.6352941176470588, 0.6352941176470588, None)
(0.5599037054261445, 0.5029082102570607, 0.4023491722353325, None)


In [49]:
#SVM
y_pred_svm = classifier_svm.predict(X_t)
cm = confusion_matrix(y_t, y_pred_svm)
print(classification_report(y_t,y_pred_svm))
print(cm)
score_svm=precision_recall_fscore_support(y_t,y_pred_svm,average='weighted')
print(score_svm)
score_svm=precision_recall_fscore_support(y_t,y_pred_svm,average='micro')
print(score_svm)
score_svm=precision_recall_fscore_support(y_t,y_pred_svm,average='macro')
print(score_svm)

              precision    recall  f1-score   support

           0       0.42      0.15      0.22       929
           1       0.64      0.88      0.74      1621

    accuracy                           0.62      2550
   macro avg       0.53      0.52      0.48      2550
weighted avg       0.56      0.62      0.55      2550

[[ 141  788]
 [ 193 1428]]
(0.56343610082871, 0.6152941176470588, 0.5545045310675792, None)
(0.6152941176470588, 0.6152941176470588, 0.6152941176470588, None)
(0.5332800103763592, 0.5163568980595773, 0.4838047093650584, None)


In [50]:
#Logistic Regression
y_pred_lr=classifier_lr.predict(X_t)
cm = confusion_matrix(y_t, y_pred_lr)
print(classification_report(y_t,y_pred_lr))
print(cm)
score_lr=precision_recall_fscore_support(y_t,y_pred_lr,average='weighted')
print(score_lr)
score_lr=precision_recall_fscore_support(y_t,y_pred_lr,average='micro')
print(score_lr)
score_lr=precision_recall_fscore_support(y_t,y_pred_lr,average='macro')
print(score_lr)

              precision    recall  f1-score   support

           0       0.35      0.31      0.33       929
           1       0.63      0.67      0.65      1621

    accuracy                           0.54      2550
   macro avg       0.49      0.49      0.49      2550
weighted avg       0.53      0.54      0.53      2550

[[ 285  644]
 [ 527 1094]]
(0.5280072747324397, 0.5407843137254902, 0.5333516869927555, None)
(0.5407843137254902, 0.5407843137254902, 0.5407843137254902, None)
(0.4902221850606835, 0.49083676370882967, 0.4893911938384605, None)


In [51]:
#Decision Tree
y_pred_dt = classifier_dt.predict(X_t)
cm = confusion_matrix(y_t, y_pred_dt)
print(classification_report(y_t,y_pred_dt))
print(cm)
score_dt=precision_recall_fscore_support(y_t,y_pred_dt,average='weighted')
print(score_dt)
score_dt=precision_recall_fscore_support(y_t,y_pred_dt,average='micro')
print(score_dt)
score_dt=precision_recall_fscore_support(y_t,y_pred_dt,average='macro')
print(score_dt)

              precision    recall  f1-score   support

           0       0.36      0.90      0.51       929
           1       0.58      0.08      0.13      1621

    accuracy                           0.38      2550
   macro avg       0.47      0.49      0.32      2550
weighted avg       0.50      0.38      0.27      2550

[[ 840   89]
 [1499  122]]
(0.498388399926754, 0.3772549019607843, 0.271950540902223, None)
(0.3772549019607843, 0.3772549019607843, 0.3772549019607843, None)
(0.46866344226985646, 0.4897301231349305, 0.3236318301593325, None)


In [52]:
#Random Florest
y_pred_rf = classifier_rf.predict(X_t)
cm = confusion_matrix(y_t, y_pred_rf)
print(classification_report(y_t,y_pred_rf))
print(cm)
score_rf=precision_recall_fscore_support(y_t,y_pred_rf,average='weighted')
print(score_rf)
score_rf=precision_recall_fscore_support(y_t,y_pred_rf,average='micro')
print(score_rf)
score_rf=precision_recall_fscore_support(y_t,y_pred_rf,average='macro')
print(score_rf)

              precision    recall  f1-score   support

           0       0.36      0.96      0.53       929
           1       0.62      0.04      0.07      1621

    accuracy                           0.37      2550
   macro avg       0.49      0.50      0.30      2550
weighted avg       0.53      0.37      0.24      2550

[[ 894   35]
 [1563   58]]
(0.5290081006210039, 0.37333333333333335, 0.23540024669728207, None)
(0.37333333333333335, 0.37333333333333335, 0.37333333333333335, None)
(0.4937571389184292, 0.49905273160596025, 0.29786732519999637, None)


## Test on Facebook

In [ ]:
d = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/ML_TEST/Data/ML_FB_Xtest.csv')
# d.head(5)
d.columns.values[0] = 'id'
d.drop('id', inplace=True, axis=1)
X_t = d.to_numpy()
de = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/ML_TEST/Data/ML_FB_ytest.csv')
# d.head(5)
de.columns.values[0] = 'id'
de.drop('id', inplace=True, axis=1)
y_t = de.to_numpy()
# X_t

In [ ]:
#Naive Bayes by Davidson or itself
y_pred_np = classifier_np.predict(X_t)
cm = confusion_matrix(y_t, y_pred_np)
print(classification_report(y_t,y_pred_np))
print(cm)
score_np=precision_recall_fscore_support(y_t,y_pred_np,average='weighted')
print(score_np)

              precision    recall  f1-score   support

           0       0.39      0.58      0.47        59
           1       0.60      0.41      0.49        90

    accuracy                           0.48       149
   macro avg       0.49      0.49      0.48       149
weighted avg       0.52      0.48      0.48       149

[[34 25]
 [53 37]]
(0.5152157629780044, 0.47651006711409394, 0.47849155388240766, None)


In [ ]:
#KNN
y_pred_knn = classifier_knn.predict(X_t)
cm = confusion_matrix(y_t, y_pred_knn)
print(classification_report(y_t,y_pred_knn))
print(cm)
score_knn=precision_recall_fscore_support(y_t,y_pred_knn,average='weighted')
print(score_knn)

              precision    recall  f1-score   support

           0       0.50      0.03      0.06        59
           1       0.61      0.98      0.75        90

    accuracy                           0.60       149
   macro avg       0.55      0.51      0.41       149
weighted avg       0.56      0.60      0.48       149

[[ 2 57]
 [ 2 88]]
(0.5645683869474658, 0.6040268456375839, 0.4775187051354408, None)


In [ ]:
#XGBoost Classifier
y_pred_xgb = classifier_xgb.predict(X_t)
cm = confusion_matrix(y_t, y_pred_xgb)
print(classification_report(y_t,y_pred_xgb))
print(cm)
score_xgb=precision_recall_fscore_support(y_t,y_pred_xgb,average='weighted')
print(score_xgb)

              precision    recall  f1-score   support

           0       0.50      0.05      0.09        59
           1       0.61      0.97      0.75        90

    accuracy                           0.60       149
   macro avg       0.55      0.51      0.42       149
weighted avg       0.57      0.60      0.49       149

[[ 3 56]
 [ 3 87]]
(0.5654714413103675, 0.6040268456375839, 0.4876272099016, None)


In [ ]:
#SVM
y_pred_svm = classifier_svm.predict(X_t)
cm = confusion_matrix(y_t, y_pred_svm)
print(classification_report(y_t,y_pred_svm))
print(cm)
score_svm=precision_recall_fscore_support(y_t,y_pred_svm,average='weighted')
print(score_svm)

              precision    recall  f1-score   support

           0       0.43      0.54      0.48        59
           1       0.64      0.53      0.58        90

    accuracy                           0.54       149
   macro avg       0.54      0.54      0.53       149
weighted avg       0.56      0.54      0.54       149

[[32 27]
 [42 48]]
(0.5578088155269363, 0.5369127516778524, 0.5419772738741301, None)


In [ ]:
#Logistic Regression
y_pred_lr=classifier_lr.predict(X_t)
cm = confusion_matrix(y_t, y_pred_lr)
print(classification_report(y_t,y_pred_lr))
print(cm)
score_lr=precision_recall_fscore_support(y_t,y_pred_lr,average='weighted')
print(score_lr)

              precision    recall  f1-score   support

           0       0.45      0.54      0.49        59
           1       0.65      0.57      0.61        90

    accuracy                           0.56       149
   macro avg       0.55      0.55      0.55       149
weighted avg       0.57      0.56      0.56       149

[[32 27]
 [39 51]]
(0.573407403637104, 0.5570469798657718, 0.561671214691349, None)


In [ ]:
#Decision Tree
y_pred_dt = classifier_dt.predict(X_t)
cm = confusion_matrix(y_t, y_pred_dt)
print(classification_report(y_t,y_pred_dt))
print(cm)
score_dt=precision_recall_fscore_support(y_t,y_pred_dt,average='weighted')
print(score_dt)

              precision    recall  f1-score   support

           0       0.39      0.88      0.54        59
           1       0.50      0.08      0.13        90

    accuracy                           0.40       149
   macro avg       0.44      0.48      0.34       149
weighted avg       0.45      0.40      0.29       149

[[52  7]
 [83  7]]
(0.4545364156102411, 0.3959731543624161, 0.2935855744615172, None)


In [ ]:
#Random Florest
y_pred_rf = classifier_rf.predict(X_t)
cm = confusion_matrix(y_t, y_pred_rf)
print(classification_report(y_t,y_pred_rf))
print(cm)
score_rf=precision_recall_fscore_support(y_t,y_pred_rf,average='weighted')
print(score_rf)

              precision    recall  f1-score   support

           0       0.38      0.90      0.54        59
           1       0.45      0.06      0.10        90

    accuracy                           0.39       149
   macro avg       0.42      0.48      0.32       149
weighted avg       0.43      0.39      0.27       149

[[53  6]
 [85  5]]
(0.42663430334863073, 0.38926174496644295, 0.2728663354530868, None)


## Test on stormfront

In [ ]:
d = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/ML_TEST/Data/ML_SF_Xtest.csv')
# d.head(5)
d.columns.values[0] = 'id'
d.drop('id', inplace=True, axis=1)
X_t = d.to_numpy()
de = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/ML_TEST/Data/ML_SF_ytest.csv')
# d.head(5)
de.columns.values[0] = 'id'
de.drop('id', inplace=True, axis=1)
y_t = de.to_numpy()
# X_t

In [ ]:
#Naive Bayes by Davidson or itself
y_pred_np = classifier_np.predict(X_t)
cm = confusion_matrix(y_t, y_pred_np)
print(classification_report(y_t,y_pred_np))
print(cm)
score_np=precision_recall_fscore_support(y_t,y_pred_np,average='weighted')
print(score_np)

              precision    recall  f1-score   support

           0       0.14      0.60      0.23       342
           1       0.92      0.57      0.70      2942

    accuracy                           0.57      3284
   macro avg       0.53      0.58      0.46      3284
weighted avg       0.84      0.57      0.65      3284

[[ 206  136]
 [1278 1664]]
(0.8426278773101561, 0.5694275274056029, 0.6522231861895078, None)


In [ ]:
#KNN
y_pred_knn = classifier_knn.predict(X_t)
cm = confusion_matrix(y_t, y_pred_knn)
print(classification_report(y_t,y_pred_knn))
print(cm)
score_knn=precision_recall_fscore_support(y_t,y_pred_knn,average='weighted')
print(score_knn)

              precision    recall  f1-score   support

           0       0.09      0.03      0.04       342
           1       0.90      0.97      0.93      2942

    accuracy                           0.87      3284
   macro avg       0.49      0.50      0.49      3284
weighted avg       0.81      0.87      0.84      3284

[[   9  333]
 [  91 2851]]
(0.8115376593687071, 0.8708891595615104, 0.8380945181978184, None)


In [ ]:
#XGBoost Classifier
y_pred_xgb = classifier_xgb.predict(X_t)
cm = confusion_matrix(y_t, y_pred_xgb)
print(classification_report(y_t,y_pred_xgb))
print(cm)
score_xgb=precision_recall_fscore_support(y_t,y_pred_xgb,average='weighted')
print(score_xgb)

              precision    recall  f1-score   support

           0       0.17      0.01      0.02       342
           1       0.90      0.99      0.94      2942

    accuracy                           0.89      3284
   macro avg       0.53      0.50      0.48      3284
weighted avg       0.82      0.89      0.85      3284

[[   3  339]
 [  15 2927]]
(0.8202284191832134, 0.892204628501827, 0.8465096704421312, None)


In [ ]:
#SVM
y_pred_svm = classifier_svm.predict(X_t)
cm = confusion_matrix(y_t, y_pred_svm)
print(classification_report(y_t,y_pred_svm))
print(cm)
score_svm=precision_recall_fscore_support(y_t,y_pred_svm,average='weighted')
print(score_svm)

              precision    recall  f1-score   support

           0       0.16      0.25      0.19       342
           1       0.91      0.84      0.87      2942

    accuracy                           0.78      3284
   macro avg       0.53      0.55      0.53      3284
weighted avg       0.83      0.78      0.80      3284

[[  86  256]
 [ 464 2478]]
(0.8282582638019074, 0.7807551766138855, 0.8023001825013596, None)


In [ ]:
#Logistic Regression
y_pred_lr=classifier_lr.predict(X_t)
cm = confusion_matrix(y_t, y_pred_lr)
print(classification_report(y_t,y_pred_lr))
print(cm)
score_lr=precision_recall_fscore_support(y_t,y_pred_lr,average='weighted')
print(score_lr)

              precision    recall  f1-score   support

           0       0.12      0.40      0.19       342
           1       0.91      0.67      0.77      2942

    accuracy                           0.64      3284
   macro avg       0.51      0.53      0.48      3284
weighted avg       0.82      0.64      0.71      3284

[[ 137  205]
 [ 980 1962]]
(0.8238826555858774, 0.6391595615103532, 0.7076277091728452, None)


In [ ]:
#Decision Tree
y_pred_dt = classifier_dt.predict(X_t)
cm = confusion_matrix(y_t, y_pred_dt)
print(classification_report(y_t,y_pred_dt))
print(cm)
score_dt=precision_recall_fscore_support(y_t,y_pred_dt,average='weighted')
print(score_dt)

              precision    recall  f1-score   support

           0       0.10      0.85      0.18       342
           1       0.87      0.11      0.20      2942

    accuracy                           0.19      3284
   macro avg       0.48      0.48      0.19      3284
weighted avg       0.79      0.19      0.20      3284

[[ 291   51]
 [2610  332]]
(0.7870132558324492, 0.1897076735688185, 0.19759186699380865, None)


In [ ]:
#Random Florest
y_pred_rf = classifier_rf.predict(X_t)
cm = confusion_matrix(y_t, y_pred_rf)
print(classification_report(y_t,y_pred_rf))
print(cm)
score_rf=precision_recall_fscore_support(y_t,y_pred_rf,average='weighted')
print(score_rf)

              precision    recall  f1-score   support

           0       0.10      0.94      0.18       342
           1       0.84      0.04      0.07      2942

    accuracy                           0.13      3284
   macro avg       0.47      0.49      0.13      3284
weighted avg       0.76      0.13      0.08      3284

[[ 322   20]
 [2836  106]]
(0.7642775001932458, 0.13032886723507917, 0.08106618474941521, None)


## Test on Reddit

In [ ]:
d = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/ML_TEST/Data/ML_RE_Xtest.csv')
# d.head(5)
d.columns.values[0] = 'id'
d.drop('id', inplace=True, axis=1)
X_t = d.to_numpy()
de = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/ML_TEST/Data/ML_RE_ytest.csv')
# d.head(5)
de.columns.values[0] = 'id'
de.drop('id', inplace=True, axis=1)
y_t = de.to_numpy()
# X_t

In [ ]:
#Naive Bayes by Davidson or itself
y_pred_np = classifier_np.predict(X_t)
cm = confusion_matrix(y_t, y_pred_np)
print(classification_report(y_t,y_pred_np))
print(cm)
score_np=precision_recall_fscore_support(y_t,y_pred_np,average='weighted')
print(score_np)

              precision    recall  f1-score   support

         0.0       0.72      0.53      0.61      5813
         1.0       0.65      0.81      0.72      6188

    accuracy                           0.68     12001
   macro avg       0.69      0.67      0.67     12001
weighted avg       0.68      0.68      0.67     12001

[[3093 2720]
 [1178 5010]]
(0.6849669097794283, 0.6751937338555121, 0.6683527082421057, None)


In [ ]:
#KNN
y_pred_knn = classifier_knn.predict(X_t)
cm = confusion_matrix(y_t, y_pred_knn)
print(classification_report(y_t,y_pred_knn))
print(cm)
score_knn=precision_recall_fscore_support(y_t,y_pred_knn,average='weighted')
print(score_knn)

              precision    recall  f1-score   support

         0.0       0.86      0.27      0.41      5813
         1.0       0.58      0.96      0.73      6188

    accuracy                           0.63     12001
   macro avg       0.72      0.62      0.57     12001
weighted avg       0.72      0.63      0.57     12001

[[1585 4228]
 [ 255 5933]]
(0.7183203408613638, 0.6264477960169986, 0.5748731214326025, None)


In [ ]:
#XGBoost Classifier
y_pred_xgb = classifier_xgb.predict(X_t)
cm = confusion_matrix(y_t, y_pred_xgb)
print(classification_report(y_t,y_pred_xgb))
print(cm)
score_xgb=precision_recall_fscore_support(y_t,y_pred_xgb,average='weighted')
print(score_xgb)

              precision    recall  f1-score   support

         0.0       0.81      0.70      0.75      5813
         1.0       0.75      0.84      0.79      6188

    accuracy                           0.77     12001
   macro avg       0.78      0.77      0.77     12001
weighted avg       0.78      0.77      0.77     12001

[[4068 1745]
 [ 967 5221]]
(0.7778078188073158, 0.7740188317640196, 0.7725982257801054, None)


In [ ]:
#SVM
y_pred_svm = classifier_svm.predict(X_t)
cm = confusion_matrix(y_t, y_pred_svm)
print(classification_report(y_t,y_pred_svm))
print(cm)
score_svm=precision_recall_fscore_support(y_t,y_pred_svm,average='weighted')
print(score_svm)

              precision    recall  f1-score   support

         0.0       0.80      0.79      0.80      5813
         1.0       0.81      0.81      0.81      6188

    accuracy                           0.80     12001
   macro avg       0.80      0.80      0.80     12001
weighted avg       0.80      0.80      0.80     12001

[[4615 1198]
 [1176 5012]]
(0.8021645279735785, 0.8021831514040496, 0.8021711421200771, None)


In [ ]:
#Logistic Regression
y_pred_lr=classifier_lr.predict(X_t)
cm = confusion_matrix(y_t, y_pred_lr)
print(classification_report(y_t,y_pred_lr))
print(cm)
score_lr=precision_recall_fscore_support(y_t,y_pred_lr,average='weighted')
print(score_lr)

              precision    recall  f1-score   support

         0.0       0.80      0.75      0.78      5813
         1.0       0.78      0.82      0.80      6188

    accuracy                           0.79     12001
   macro avg       0.79      0.79      0.79     12001
weighted avg       0.79      0.79      0.79     12001

[[4382 1431]
 [1093 5095]]
(0.790237397564223, 0.789684192983918, 0.7893310269427718, None)


In [ ]:
#Decision Tree
y_pred_dt = classifier_dt.predict(X_t)
cm = confusion_matrix(y_t, y_pred_dt)
print(classification_report(y_t,y_pred_dt))
print(cm)
score_dt=precision_recall_fscore_support(y_t,y_pred_dt,average='weighted')
print(score_dt)

              precision    recall  f1-score   support

         0.0       0.73      0.71      0.72      5813
         1.0       0.73      0.75      0.74      6188

    accuracy                           0.73     12001
   macro avg       0.73      0.73      0.73     12001
weighted avg       0.73      0.73      0.73     12001

[[4133 1680]
 [1557 4631]]
(0.7301962637059506, 0.7302724772935588, 0.730157563190923, None)


In [ ]:
#Random Florest
y_pred_rf = classifier_rf.predict(X_t)
cm = confusion_matrix(y_t, y_pred_rf)
print(classification_report(y_t,y_pred_rf))
print(cm)
score_rf=precision_recall_fscore_support(y_t,y_pred_rf,average='weighted')
print(score_rf)

              precision    recall  f1-score   support

         0.0       0.75      0.78      0.76      5813
         1.0       0.78      0.75      0.77      6188

    accuracy                           0.76     12001
   macro avg       0.76      0.77      0.76     12001
weighted avg       0.77      0.76      0.76     12001

[[4511 1302]
 [1522 4666]]
(0.7653116765573986, 0.7646862761436547, 0.7647419997500496, None)


## Test on Davidson

In [ ]:
d = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/ML_TEST/Data/ML_DAV_Xtest.csv')
# d.head(5)
d.columns.values[0] = 'id'
d.drop('id', inplace=True, axis=1)
X_t = d.to_numpy()
de = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/ML_TEST/Data/ML_DAV_ytest.csv')
# d.head(5)
de.columns.values[0] = 'id'
de.drop('id', inplace=True, axis=1)
y_t = de.to_numpy()
# X_t

In [ ]:
#Naive Bayes by Davidson or itself
y_pred_np = classifier_np.predict(X_t)
cm = confusion_matrix(y_t, y_pred_np)
print(classification_report(y_t,y_pred_np))
print(cm)
score_np=precision_recall_fscore_support(y_t,y_pred_np,average='weighted')
print(score_np)

              precision    recall  f1-score   support

           0       0.06      0.50      0.11       427
           1       0.94      0.52      0.67      7008

    accuracy                           0.52      7435
   macro avg       0.50      0.51      0.39      7435
weighted avg       0.89      0.52      0.64      7435

[[ 213  214]
 [3382 3626]]
(0.893443080644211, 0.5163416274377942, 0.636200014370084, None)


In [ ]:
#KNN
y_pred_knn = classifier_knn.predict(X_t)
cm = confusion_matrix(y_t, y_pred_knn)
print(classification_report(y_t,y_pred_knn))
print(cm)
score_knn=precision_recall_fscore_support(y_t,y_pred_knn,average='weighted')
print(score_knn)

              precision    recall  f1-score   support

           0       0.06      0.02      0.03       427
           1       0.94      0.98      0.96      7008

    accuracy                           0.92      7435
   macro avg       0.50      0.50      0.50      7435
weighted avg       0.89      0.92      0.91      7435

[[  10  417]
 [ 168 6840]]
(0.8916337246198329, 0.9213180901143242, 0.9058132321066279, None)


In [ ]:
#XGBoost Classifier
y_pred_xgb = classifier_xgb.predict(X_t)
cm = confusion_matrix(y_t, y_pred_xgb)
print(classification_report(y_t,y_pred_xgb))
print(cm)
score_xgb=precision_recall_fscore_support(y_t,y_pred_xgb,average='weighted')
print(score_xgb)

              precision    recall  f1-score   support

           0       0.06      0.01      0.02       427
           1       0.94      0.99      0.97      7008

    accuracy                           0.93      7435
   macro avg       0.50      0.50      0.49      7435
weighted avg       0.89      0.93      0.91      7435

[[   5  422]
 [  78 6930]]
(0.8919257924646022, 0.9327505043712172, 0.9108757776675623, None)


In [ ]:
#SVM
y_pred_svm = classifier_svm.predict(X_t)
cm = confusion_matrix(y_t, y_pred_svm)
print(classification_report(y_t,y_pred_svm))
print(cm)
score_svm=precision_recall_fscore_support(y_t,y_pred_svm,average='weighted')
print(score_svm)

              precision    recall  f1-score   support

           0       0.06      0.19      0.09       427
           1       0.94      0.81      0.87      7008

    accuracy                           0.77      7435
   macro avg       0.50      0.50      0.48      7435
weighted avg       0.89      0.77      0.83      7435

[[  82  345]
 [1341 5667]]
(0.8917888465165962, 0.7732347007397444, 0.8256039563374964, None)


In [ ]:
#Logistic Regression
y_pred_lr=classifier_lr.predict(X_t)
cm = confusion_matrix(y_t, y_pred_lr)
print(classification_report(y_t,y_pred_lr))
print(cm)
score_lr=precision_recall_fscore_support(y_t,y_pred_lr,average='weighted')
print(score_lr)

              precision    recall  f1-score   support

           0       0.05      0.42      0.09       427
           1       0.94      0.53      0.68      7008

    accuracy                           0.53      7435
   macro avg       0.50      0.48      0.39      7435
weighted avg       0.89      0.53      0.65      7435

[[ 180  247]
 [3274 3734]]
(0.8870804460698115, 0.5264290517821116, 0.6458864579584789, None)


In [ ]:
#Decision Tree
y_pred_dt = classifier_dt.predict(X_t)
cm = confusion_matrix(y_t, y_pred_dt)
print(classification_report(y_t,y_pred_dt))
print(cm)
score_dt=precision_recall_fscore_support(y_t,y_pred_dt,average='weighted')
print(score_dt)

              precision    recall  f1-score   support

           0       0.06      0.90      0.11       427
           1       0.93      0.08      0.16      7008

    accuracy                           0.13      7435
   macro avg       0.49      0.49      0.13      7435
weighted avg       0.88      0.13      0.15      7435

[[ 383   44]
 [6413  595]]
(0.8809025301843406, 0.13154001344989913, 0.15276992663949177, None)


In [ ]:
#Random Florest
y_pred_rf = classifier_rf.predict(X_t)
cm = confusion_matrix(y_t, y_pred_rf)
print(classification_report(y_t,y_pred_rf))
print(cm)
score_rf=precision_recall_fscore_support(y_t,y_pred_rf,average='weighted')
print(score_rf)

              precision    recall  f1-score   support

           0       0.06      0.96      0.11       427
           1       0.94      0.04      0.08      7008

    accuracy                           0.09      7435
   macro avg       0.50      0.50      0.09      7435
weighted avg       0.89      0.09      0.08      7435

[[ 410   17]
 [6730  278]]
(0.8915492616523887, 0.09253530598520511, 0.07798422422075785, None)


## Test on Waseem

In [ ]:
d = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/ML_TEST/Data/ML_WS_Xtest.csv')
# d.head(5)
d.columns.values[0] = 'id'
d.drop('id', inplace=True, axis=1)
X_t = d.to_numpy()
de = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/ML_TEST/Data/ML_WS_ytest.csv')
# d.head(5)
de.columns.values[0] = 'id'
de.drop('id', inplace=True, axis=1)
y_t = de.to_numpy()
# X_t

In [ ]:
#Naive Bayes by Davidson or itself
y_pred_np = classifier_np.predict(X_t)
cm = confusion_matrix(y_t, y_pred_np)
print(classification_report(y_t,y_pred_np))
print(cm)
score_np=precision_recall_fscore_support(y_t,y_pred_np,average='weighted')
print(score_np)

              precision    recall  f1-score   support

           0       0.32      0.63      0.43       898
           1       0.80      0.52      0.63      2500

    accuracy                           0.55      3398
   macro avg       0.56      0.58      0.53      3398
weighted avg       0.67      0.55      0.58      3398

[[ 568  330]
 [1198 1302]]
(0.6719569617948212, 0.5503237198351971, 0.5763505472018025, None)


In [ ]:
#KNN
y_pred_knn = classifier_knn.predict(X_t)
cm = confusion_matrix(y_t, y_pred_knn)
print(classification_report(y_t,y_pred_knn))
print(cm)
score_knn=precision_recall_fscore_support(y_t,y_pred_knn,average='weighted')
print(score_knn)

              precision    recall  f1-score   support

           0       0.27      0.03      0.05       898
           1       0.74      0.97      0.84      2500

    accuracy                           0.72      3398
   macro avg       0.50      0.50      0.44      3398
weighted avg       0.61      0.72      0.63      3398

[[  24  874]
 [  65 2435]]
(0.6126653810770893, 0.7236609770453207, 0.6296519797335509, None)


In [ ]:
#XGBoost Classifier
y_pred_xgb = classifier_xgb.predict(X_t)
cm = confusion_matrix(y_t, y_pred_xgb)
print(classification_report(y_t,y_pred_xgb))
print(cm)
score_xgb=precision_recall_fscore_support(y_t,y_pred_xgb,average='weighted')
print(score_xgb)

              precision    recall  f1-score   support

           0       0.26      0.02      0.03       898
           1       0.74      0.98      0.84      2500

    accuracy                           0.73      3398
   macro avg       0.50      0.50      0.44      3398
weighted avg       0.61      0.73      0.63      3398

[[  14  884]
 [  40 2460]]
(0.6097497490731901, 0.7280753384343731, 0.6271732069787185, None)


In [ ]:
#SVM
y_pred_svm = classifier_svm.predict(X_t)
cm = confusion_matrix(y_t, y_pred_svm)
print(classification_report(y_t,y_pred_svm))
print(cm)
score_svm=precision_recall_fscore_support(y_t,y_pred_svm,average='weighted')
print(score_svm)

              precision    recall  f1-score   support

           0       0.44      0.34      0.38       898
           1       0.78      0.85      0.81      2500

    accuracy                           0.71      3398
   macro avg       0.61      0.59      0.60      3398
weighted avg       0.69      0.71      0.70      3398

[[ 302  596]
 [ 383 2117]]
(0.6906118452006457, 0.7118893466745144, 0.6983920463829955, None)


In [ ]:
#Logistic Regression
y_pred_lr=classifier_lr.predict(X_t)
cm = confusion_matrix(y_t, y_pred_lr)
print(classification_report(y_t,y_pred_lr))
print(cm)
score_lr=precision_recall_fscore_support(y_t,y_pred_lr,average='weighted')
print(score_lr)

              precision    recall  f1-score   support

           0       0.35      0.56      0.43       898
           1       0.80      0.63      0.70      2500

    accuracy                           0.61      3398
   macro avg       0.58      0.59      0.57      3398
weighted avg       0.68      0.61      0.63      3398

[[ 504  394]
 [ 930 1570]]
(0.6810145873611407, 0.6103590347263096, 0.6317452469872156, None)


In [ ]:
#Decision Tree
y_pred_dt = classifier_dt.predict(X_t)
cm = confusion_matrix(y_t, y_pred_dt)
print(classification_report(y_t,y_pred_dt))
print(cm)
score_dt=precision_recall_fscore_support(y_t,y_pred_dt,average='weighted')
print(score_dt)

              precision    recall  f1-score   support

           0       0.26      0.93      0.41       898
           1       0.68      0.05      0.10      2500

    accuracy                           0.28      3398
   macro avg       0.47      0.49      0.25      3398
weighted avg       0.57      0.28      0.18      3398

[[ 838   60]
 [2372  128]]
(0.5699113632202599, 0.2842848734549735, 0.17788853673700308, None)


In [ ]:
#Random Florest
y_pred_rf = classifier_rf.predict(X_t)
cm = confusion_matrix(y_t, y_pred_rf)
print(classification_report(y_t,y_pred_rf))
print(cm)
score_rf=precision_recall_fscore_support(y_t,y_pred_rf,average='weighted')
print(score_rf)

              precision    recall  f1-score   support

           0       0.26      0.95      0.41       898
           1       0.70      0.04      0.07      2500

    accuracy                           0.28      3398
   macro avg       0.48      0.50      0.24      3398
weighted avg       0.58      0.28      0.16      3398

[[ 856   42]
 [2404   96]]
(0.5812019716931383, 0.2801648028251913, 0.16235883608967092, None)
